# Decision Trees Exercise
In this exercise you will show that ID3 is sub-optimal. Implement a simple version of Decision Tree, and will then apply a Decision Tree classsifier on the MNIST hand written digits dataset that we already saw.


## 1. Suboptimality of ID3
Consider the following training set, where $\mathcal{X} = \{0, 1\}^3$ and $\mathcal{Y} =\{0, 1\}$:

$$
\begin{aligned}
((1, 1, 1), 1)\\
((1, 0, 0), 1)\\
((1, 1, 0), 0)\\
((0, 0, 1), 0)
\end{aligned}
$$

Suppose we wish to use this training set in order to build a decision tree of depth 2 (i.e. for each
input we are allowed to ask two questions of the form "$x_i = 0$?" before deciding on the label).

1. Suppose we run the ID3 algorithm up to depth 2 (namely, we pick the root node and its
children according to the algorithm, but instead of keeping on with the recursion, we stop
and pick leaves according to the majority label in each subtree, once we reach depth 2). 
Assume that the subroutine used to measure the quality of each feature is based on the information gain, and that if two features get the same score, one of them is picked arbitrarily. 
Show that the training error of the resulting decision tree is at least 1/4.
2. Find a decision tree of depth 2, which attains zero training error.


#### Answer
![alt text](https://snipboard.io/DSFTtz.jpg)

#### Answer
![alt text](https://snipboard.io/DSFTtz.jpg)

In [ ]:
class NODE:
   def __init__(self,feature,value,L=None,R=None):
    self.feature = feature
    self.value = value
    self.L = L
    self.R = R


def fit():
    get_f_v()
    tree = NODE(f,v)
    pass

In [57]:
np.sqrt( (9+9+36)) /4

1.8371173070873836

## 2. Implementing Decision Tree From Scratch
In this exercise you will need to implement a simple version of Decision Tree from scratch. Your decision tree will handle **continuous input and output** (this should actually work also for binary input attributes).

* Compelete the skeleton class below
  - `X` is a matrix of data values (rows are samples, columns are attributes)
  - `y` is a vector of corresponding target values
  - `min_leaf` is the minimal number of samples in each leaf node
  
* For splitting criterion, use either **"Train Squared Error Minimization (Reduction in Variance)"** or **"Train Absolute Error Minimization"** (choose one). Whatever you choose, make sure you implement the splitting point decision efficiently (in $O(nlgn)$ time).

* The `predict` function will use mean of the target values in the leaf node matching each row of the given `X`. The result is a vector of predictions matching the number of rows in `X`.

* To check your decision tree implementation, use the boston dataset (`from sklearn.datasets import load_boston`) split the data set into train and test using (`from sklearn.model_selection import train_test_split`)

  - Use the following to estimate what are the best hyper parameters to use for your model
```
    for min_leaf in [1,5,10,100]:
      dt = DecisionTree(X, y, n, sz, min_leaf)
      mse = # mean square error over test set
      print("min_leaf:{0} --- oob mse: {1}".format(min_leaf, mse))
```
  
  - Using your chosen hyperparameters as a final model, plot the predictions vs. true values of all the samples in the training set . Use something like:
  ```
  y_hat = dt.predict(X_train)  # forest is the chosen model
  plt.scatter(y_hat, y_test)
  ```

In [41]:
# Libraries
import pandas as pd
from sklearn.datasets import load_boston
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [57]:
# Load the Boston dataset
boston_dataset = load_boston()


# Convert the dataset into a pandas dataframe
boston_df = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston_df['target'] = boston_dataset.target
from sklearn.model_selection import train_test_split
y = boston_df['target']
X = boston_df.drop(['target'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Calc RMSE


In [8]:
### # FOR DEBUGGING
def find_feat_val(df,y):
    RMSE = np.inf
    VAL = 0
    COL = ''
    for colname, column in df.iteritems():
        rmse,val = calc_rmse(column,y)
        
        if RMSE > rmse:
            RMSE = rmse
            VAL = val
            COL = colname

    return VAL,COL
    

def calc_rmse(X,y):
    # concatenate feature and target to easily sort
    df = pd.concat([X,y],axis=1)
    df.sort_values(df.columns[0],inplace=True)

    # Create different splits of data and find the one with least RMSE
    
    best_rmse = np.inf

    for indx in range(df.shape[0]-1): ## CAN BE IMPROVED BY ONLY TAKE UNIQUE VALUES OF X
        left = df.iloc[:indx+1]
        right = df.iloc[indx+1:]
        left_mean = left.iloc[:,-1].mean()
        right_mean = right.iloc[:,-1].mean()
        l_rmse = np.sqrt((left.iloc[:,-1] - left_mean)**2).mean()
        r_rmse = np.sqrt((right.iloc[:,-1] - right_mean)**2).mean()
        t_rmse = r_rmse * right.shape[0]/df.shape[0] + l_rmse * left.shape[0]/df.shape[0]# Total RMSE

        if best_rmse > t_rmse: # if found a better rmse
            best_rmse = t_rmse
            split_val = df.iloc[indx,0]

    
    return best_rmse,split_val

# def calc_rmse(X,y): # OF chatGPT
#     # concatenate feature and target to easily sort
#     df = pd.concat([X,y],axis=1)
#     df.sort_values(df.columns[0],inplace=True)

#     # Create different splits of data and find the one with least RMSE

#     best_rmse = np.inf
#     unique_x = pd.unique(df[df.columns[0]])
#     for indx in unique_x:
#         left = df[df[df.columns[0]]<=indx]
#         right = df[df[df.columns[0]]>indx]
#         left_mean = left.iloc[:,-1].mean()
#         right_mean = right.iloc[:,-1].mean()
#         l_rmse = np.sqrt((left.iloc[:,-1] - left_mean)**2).mean()
#         r_rmse = np.sqrt((right.iloc[:,-1] - right_mean)**2).mean()
#         t_rmse = r_rmse * right.shape[0]/df.shape[0] + l_rmse * left.shape[0]/df.shape[0]# Total RMSE

#         if best_rmse > t_rmse: # if found a better rmse
#             best_rmse = t_rmse
#             split_val = indx

#     return best_rmse,split_val


class NODE(object):
    def __init__(self,feature,value,L=None,R=None):
        self.feature = feature
        self.value = value
        self.L = L
        self.R = R
        ## REPLACE WITH INDICES
        self.n_samples = []
        self.mean = 0
        self.is_root = False

In [98]:
class DecisionTree():
  def __init__(self, X, y, min_leaf=20):
    self.X = X
    self.y = y
    self.min_leaf = min_leaf
    self.Tree = None

  class NODE(object):
    def __init__(self,feature,value,L=None,R=None):
      self.feature = feature
      self.value = value
      self.L = L
      self.R = R
      self.n_samples = []
      self.mean = 0
      self.is_root = False
    



  def find_feat_val(self,df,y):
      RMSE = np.inf
      VAL = 0
      COL = ''
      for colname, column in df.iteritems():
          rmse,val = self.calc_rmse(column,y)
          
          if RMSE > rmse:
              RMSE = rmse
              VAL = val
              COL = colname

      return VAL,COL
    

  def calc_rmse(self,X,y):
      # concatenate feature and target to easily sort
      df = pd.concat([X,y],axis=1)
      df.sort_values(df.columns[0],inplace=True)

      # Create different splits of data and find the one with least RMSE
      
      best_rmse = np.inf

      for indx in range(df.shape[0]-1): ## CAN BE IMPROVED BY ONLY TAKE UNIQUE VALUES OF X
          left = df.iloc[:indx+1]
          right = df.iloc[indx+1:]
          left_mean = left.iloc[:,-1].mean()
          right_mean = right.iloc[:,-1].mean()
          l_rmse = np.sqrt((left.iloc[:,-1] - left_mean)**2).mean()
          r_rmse = np.sqrt((right.iloc[:,-1] - right_mean)**2).mean()
          t_rmse = r_rmse * right.shape[0]/df.shape[0] + l_rmse * left.shape[0]/df.shape[0]# Total RMSE

          if best_rmse > t_rmse: # if found a better rmse
              best_rmse = t_rmse
              split_val = df.iloc[indx,0]

      
      return best_rmse,split_val



  def fit(self,X,y):
    # Find best feature to split on and the value

    # split_val,feature = self.find_feat_val(X,y) # REAL
    split_val,feature = find_feat_val(X,y) # DEBUG
    
    # Create new node
    if self.Tree is None:
      
      # node = self.NODE(feature,split_val) # create new node REAL
      node = NODE(feature,split_val) # create new node DEBUG
      node.is_root = True
      self.Tree = True
      node.mean = np.mean(y)
    else:
      if feature:
        # node = self.NODE(feature,split_val) # REAL
        node = NODE(feature,split_val) # DEBUG
        node.mean = np.mean(y)
      else: 
        return


    # Create left and right data and target subsets
    
    subset_l = X[X[feature] <= split_val].drop([feature],axis=1)
    
    y_l = self.y.loc[subset_l.index]
    
    subset_r = X[X[feature] > split_val].drop([feature],axis=1)
    y_r = self.y.loc[subset_r.index]
   
   


    # Check that there are features and samples to split node in left subset
    if (subset_l.shape[0] >= self.min_leaf) & (subset_l.shape[1] > 0): # Check if creating left node is possible  
      node.L = self.fit(subset_l,y_l) # below threshold

    # else: 
    #   node.n_samples.append(y_l.values)
    #   node.mean = np.mean(np.concatenate(node.n_samples))
      
      
# Check that there are features and samples to split node in right subset
    if (subset_r.shape[0] >= self.min_leaf) & (subset_r.shape[1] > 0): 
      node.R = self.fit(subset_r,y_r) # below threshold
    # else:
    #   node.n_samples.append(y_r.values)
    #   node.mean = np.mean(np.concatenate(node.n_samples))
      
           

    
    if node.is_root:
      self.Tree = node
    else: # node is leaf
      # node.mean = np.mean(np.concatenate(node.n_samples)) ## MIGHT BE RDEUNDENT
      return node

  def predict(self, X):
    y_pred = []
    
    for indx,row in X.iterrows():
      curr_node = self.Tree
      
      can_go_deep = True
      while can_go_deep: # while curr_node is not a leaf

        if row[curr_node.feature] <= curr_node.value and curr_node.L: # if equal or below splitting value
          curr_node = curr_node.L
          
        elif row[curr_node.feature] > curr_node.value and curr_node.R:
          curr_node = curr_node.R
          

        else: 
          can_go_deep = False ## break if node is a leaf

      y_pred.append(curr_node.mean)   
    
    return y_pred

min_leaf = 20
samples = X_train.shape[0]
tree = DecisionTree(X_train,y_train,min_leaf)
tree.fit(X_train, y_train)


In [102]:
y_pred = tree.predict(X_test)

In [100]:
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

# Create an instance of the DecisionTreeClassifier
clf = DecisionTreeRegressor(min_samples_leaf=20)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred_sk = clf.predict(X_test)




In [103]:
print(mean_squared_error(y_pred,y_test))
print(mean_squared_error(y_pred_sk,y_test))

25.692560791329345
19.009901095819135


In [15]:
data = {'x1': [1, 2, 3, 4, 5,1,1,1],
        'x2': [5,1,3,2,4,5,5,5],
        'x3': [10000,1134,500,100,5000,5,5,5],
        'y': [1,2,3,40,50,1,1,1]}
df = pd.DataFrame(data)



X = df[['x1', 'x2','x3']]
y = df['y']

[1.0, 2.5, 2.5, 45.0, 45.0, 1.0, 1.0, 1.0]

## 3. Using Decision Tree for Digits Classification
Remeber the MNIST dataset used - you will now test the power of decision trees on this problem.
This time you are given a free hand in choosing the test and train set sizes, model parameters (such as gain function and constraints over the trees) and features (whether to use binary pixel values or the original continous gray value).
  - You can use `sklearn.tree.DecisionTreeClassifier`
- Once you are satisfied with the model parameters, plot the importance of each of the pixels to the final decision.
- Last, estimate the class assignment probabilities for all the correctly classified and misclassified examples in your test data.
- Discuss your results.

In [ ]:
# code and answer go here